<a href="https://colab.research.google.com/github/MosaicGroupCMU/African-MRS-Tutorials/blob/main/Google-Colab/5_Quantum_Espresso_H2O_relax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to geometry optimization of molecular water using Quantum ESPRESSO

Contributors: [Seda Oturak](https://github.com/sedaoturak), [Ismaila Dabo](https://scholar.google.com/citations?user=rN299m0AAAAJ&hl=en), [Jessica Wen](https://github.com/JessicaWen-PhD), [Cierra Chandler](https://github.com/Cierra-Chandler), [Henry Eya](https://github.com/Henrynweya)



# Step 1: Install libraries, environment, and Quantum ESPRESSO

This part installs libraries for numerical calculations and plotting.

In [1]:
# load plotting libraries
import matplotlib.pyplot as plt

# load numerical libraries
import numpy as np

Quantum ESPRESSO is a plane wave code, which uses Fourier transforms to solve equations in plane wave space. This part installs libraries for fast Fourier transforms (FFTs).

In [2]:
# eliminate text output during installation
%%capture

# install mathematical libraries to peform fast Fourier transforms
# (the exclamation mark means that the command is run under Linux)
! apt-get install -y libfftw3-3 libfftw3-dev libfftw3-doc

The Atomic Simulation Environment (ASE) is a set of tools for running, visualizing, and analyzing simulations. This part installs ASE.

In [3]:
# eliminate text output during installation
%%capture

# install the Atomic simulation environment
# ! apt install ase
! pip install git+https://gitlab.com/ase/ase

Compiling Quantum ESPRESSO from scratch would take a long time. This part uploads pre-compiled executable files (`.x` extension) and additional files containing the pseudopotentials.

In [4]:
# eliminate text output during installation
%%capture

# navigate to main directory named '/content/'
%cd /content/

# download the pre-compiled files in compressed format (under Linux)
#! wget 'https://docs.google.com/uc?export=download&id=1kw_CJMjP6ggDZXDNp5phAqCPpoe2WXCA' -O qe-lite.tgz
!gdown 'https://drive.google.com/uc?export=download&id=13l-Kiyg-F6aYb5lF8M3RsE1hSnLRdGna' -O qe-lite.tgz

# unpack the compressed files (under Linux)
! tar -xvzf qe-lite.tgz

# clean up some files
! rm -rf sample_data qe-lite.tgz

# Step 2: Prepare Quantum ESPRESSO input file for SCF

The calculation is for a water molecule. The definition of the input parameters of the `pw.x` executable can be found at `www.quantum-espresso.org/Doc/INPUT_PW.html`.

Things to note for molecules:
1. The molecule should be put in a vacuum
2. KPOINTS of 1 1 1 is used

In [48]:
from ase.visualize import view
from ase.build import molecule
from ase.data.pubchem import pubchem_atoms_search
from ase import Atom
import re

#Build molecule via PubChem
water = pubchem_atoms_search(cid=962)
vac = 10
water.center(vacuum = vac)

#Build via ASE
water = molecule('H2O')
water.set_cell((10,10,10))
water.center(vacuum=None)
positions = water.get_positions()
symbols = water.get_chemical_symbols()

# Section to updated the symbols and positions into the input file
updated_positions = []

# Iterate and prepend positions with symbols
for i, pos in enumerate(positions):
    updated_positions.append(f"{symbols[i]} {pos}")

# Expression to move the element inside the brackets
pattern = r"([A-Za-z]) \[([^\]]+)\]"
updated_data = [re.sub(pattern, r"[\1 \2]", item) for item in updated_positions]

#View the cell
view(water, viewer='x3d')


[O 5.        5.        5.2981545]
[H 5.        5.763239  4.7018455]
[H 5.        4.236761  4.7018455]


In [54]:
import re

qe_input = """
&control
  calculation='relax',
  prefix='h2o',
  pseudo_dir = '/content/qe-lite/pseudopotentials',
  outdir='/content/water',
  etot_conv_thr = 1e-5,
  forc_conv_thr = 1e-4
/
&system
  ibrav = 0,
  nat = 3,
  ntyp = 2,
  ecutwfc = 30.0,
  ecutrho = 240.0,
  occupations  = 'smearing',
  degauss      = 0.001,
  smearing     = 'marzari-vanderbilt'
/
&electrons
  conv_thr = 1e-8
/
&ions
  ion_dynamics = 'bfgs'
/
ATOMIC_SPECIES
O 15.999 O.upf
H 1.008 H.upf

K_POINTS automatic
1 1 1 0 0 0

ATOMIC_POSITIONS angstrom
O 0.0000000000 0.0000000000 0.2981545000
H 0.0000000000 0.7632390000 -0.2981545000
H 0.0000000000 -0.7632390000 -0.2981545000

CELL_PARAMETERS angstrom
10.00000000000000 0.00000000000000 0.00000000000000
0.00000000000000 10.00000000000000 0.00000000000000
0.00000000000000 0.00000000000000 10.00000000000000


"""

updated_data_cleaned = [data.strip('[]') for data in updated_data]

# Combine the cleaned data into a single string with each position on a new line
updated_positions = "\n".join(updated_data_cleaned)

# Extract the part before and after the ATOMIC_POSITIONS section
start_pos_index = qe_input.find("ATOMIC_POSITIONS angstrom") + len("ATOMIC_POSITIONS angstrom")
end_pos_index = qe_input.find("CELL_PARAMETERS angstrom", start_pos_index)

# Split the input string into header (before ATOMIC_POSITIONS) and footer (after ATOMIC_POSITIONS)
header = qe_input[:start_pos_index].strip()
footer = qe_input[end_pos_index:].strip()

# Rebuild the input with updated atomic positions (without brackets)
updated_qe_input = f"{header}\n{updated_positions}\n{footer}"

# Write the updated input to a file
with open("h2o.relax.in", "w") as f:
    f.write(updated_qe_input)

# Print the content of the input file to verify
! cat h2o.relax.in


&control
  calculation='relax',
  prefix='h2o',
  pseudo_dir = '/content/qe-lite/pseudopotentials',
  outdir='/content/water',
  etot_conv_thr = 1e-5,
  forc_conv_thr = 1e-4
/
&system
  ibrav = 0,
  nat = 3,
  ntyp = 2,
  ecutwfc = 30.0,
  ecutrho = 240.0,
  occupations  = 'smearing',
  degauss      = 0.001,
  smearing     = 'marzari-vanderbilt'
/
&electrons
  conv_thr = 1e-8
/
&ions
  ion_dynamics = 'bfgs'
/
ATOMIC_SPECIES
O 15.999 O.upf
H 1.008 H.upf

K_POINTS automatic
1 1 1 0 0 0

ATOMIC_POSITIONS angstrom
O 5.        5.        5.2981545
H 5.        5.763239  4.7018455
H 5.        4.236761  4.7018455
CELL_PARAMETERS angstrom
10.00000000000000 0.00000000000000 0.00000000000000
0.00000000000000 10.00000000000000 0.00000000000000
0.00000000000000 0.00000000000000 10.00000000000000

Use ASE tools to extract information from Quantum ESPRESSO input and visualize the crystal.

In [55]:
import ase.io.espresso
from ase import Atoms
from ase.visualize import view
from ase.build import make_supercell
from ase.build import bulk

# extract unit cell information from input file using ASE
input_file = ase.io.espresso.read_espresso_in('h2o.relax.in')
water = Atoms(input_file)

# create a supercell (2 × 2 × 2) using ASE
multiplier = np.identity(3) * 2
water_supercell = make_supercell(water,multiplier)

#visualize unit supercell
view(water_supercell, viewer='x3d')


# Step 3: Run Quantum ESPRESSO using input file

In [56]:
# run the pw.x executable using si.scf.in to create si.scf.out
! /content/qe-lite/bin/pw.x < h2o.relax.in > h2o.relax.out

# print the content of the output file (under Linux)
! cat h2o.relax.out


     Program PWSCF v.7.4 starts on 15Dec2024 at 10:16:26 

     This program is part of the open-source Quantum ESPRESSO suite
     for quantum simulation of materials; please cite
         "P. Giannozzi et al., J. Phys.:Condens. Matter 21 395502 (2009);
         "P. Giannozzi et al., J. Phys.:Condens. Matter 29 465901 (2017);
         "P. Giannozzi et al., J. Chem. Phys. 152 154105 (2020);
          URL http://www.quantum-espresso.org", 
     in publications or presentations arising from this work. More details at
     http://www.quantum-espresso.org/quote

     Serial version
     8525 MiB available memory on the printing compute node when the environment starts

     Waiting for input...
     Reading input from standard input

     Current dimensions of program PWSCF are:
     Max number of different atomic species (ntypx) = 10
     Max number of k-points (npk) =  40000
     Max angular momentum in pseudopotentials (lmaxx) =  4
     file O.upf: wavefunction(s)  2P renormalized
    

#Step 4: Extract the total energy of the system

In [ ]:
# First method: xtract total energies in rydberg during the self-consistent-field calculation (under Linux)
! grep -e "total energy  " -e "estimated" h2o.relax.out #NOTE: grep is a power comand that can seacrh and list the specified term in a file

     total energy              =     -43.80322895 Ry
     estimated scf accuracy    <       0.40856669 Ry
     total energy              =     -43.89585675 Ry
     estimated scf accuracy    <       0.37041931 Ry
     total energy              =     -43.96993004 Ry
     estimated scf accuracy    <       0.00424009 Ry
     total energy              =     -43.97070127 Ry
     estimated scf accuracy    <       0.00075070 Ry
     total energy              =     -43.97066512 Ry
     estimated scf accuracy    <       0.00012596 Ry
     total energy              =     -43.97069060 Ry
     estimated scf accuracy    <       0.00000195 Ry
     total energy              =     -43.97069075 Ry
     estimated scf accuracy    <       0.00000027 Ry
     total energy              =     -43.97069071 Ry
     estimated scf accuracy    <       0.00000012 Ry
!    total energy              =     -43.97069073 Ry
     estimated scf accuracy    <          2.8E-10 Ry
     total energy              =     -43.97063

In [58]:
# define physical constants for unit conversion
from scipy.constants import physical_constants
ha_in_ev = physical_constants["Hartree energy in eV"][0]
ry_in_ev = ha_in_ev / 2.

# second method: extract total energy at the end of the self-consistent calculation (using ASE)
output = ase.io.read("/content/water/h2o.relax.out")
total_energy = output.get_total_energy()
print("Energy = %.8f eV " % total_energy)
print("Energy = %.8f Ry " % ( total_energy / ry_in_ev ) )

Energy = -598.25630420 eV 
Energy = -43.97102733 Ry 


In [59]:
from ase.io.espresso import read_espresso_out

output = read_espresso_out("h2o.relax.out")

<generator object read_espresso_out at 0x7947ce678430>
